In [1]:
#import seaborn

In [ ]:
print("Test!")

In [1]:
# TODO: Aduth file somehow?
import ee
import numpy as np
import geopandas
import matplotlib.pyplot as plt

# from bqplot import pyplot as plt
# from ipyleaflet import *

# ee.Authenticate();
# ee.Initialize();

import geemap
import pandas as pd


In [2]:
Map = geemap.Map()
latitude = 29.6487
longitude = 91.1319

center = ee.Geometry.Point(longitude, latitude)
Map.centerObject(center, 10)
Map.set_options(mapTypeId='SATELLITE');
Map

Map(center=[29.6487, 91.1319], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [3]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.set_options(mapTypeId='SATELLITE');
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [4]:
shp_path = './Lhasa/Lhasa_RC_DGO2km_updated.shp';
lhasa_shp = geemap.shp_to_ee(shp_path);

In [5]:
Map

Map(bottom=1851.0, center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [6]:
#fait apparaitre la rivière avec les DGOs
def getCover(image, AOI, scale):
    totPixels = ee.Number(image.unmask(1).reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    actPixels = ee.Number(image.reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    percCover = actPixels.divide(totPixels).multiply(100).round();
    return image.set('percCover', percCover);

def maskL8sr(image):
    # AT the DGO SCALE we calculate n.o. pixels within DGO
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    qa = image.select('QA_PIXEL');
    mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)));
    return image.updateMask(mask);

def addMNDWI(image):
    return image.addBands(srcImg=image.normalizedDifference(['B2','B5']).rename('MNDWI')); # B2=greenl B4=SWIR

def addIS_WATER_MNDWI(image):
    return image.addBands(image.expression('MNDWI >  0.0', {
            'MNDWI': image.select('MNDWI'),
            }).rename('IS_WATER_MNDWI'));
def IS_WATER_INDIC(INDIC, ImColl):
    return ImColl.select('IS_WATER_' + INDIC).map(lambda image: image.updateMask(image.neq(0)));

def process_polygon(shp, dgo=1, START_DATE='1988-04-06', END_DATE='1988-04-08'):
    filt = ee.Filter.inList('DGO_FID', [dgo]);
    AOI = shp.filter(filt);
    filtered_DGO = AOI;
    AOI_ID=f'middle_ID{dgo}';

    START = ee.Date(START_DATE);
    END = ee.Date(END_DATE);
    DAY_DIFF = ee.Date(END_DATE).difference(START, 'day');
    DAY_RANGE = ee.List.sequence(0,DAY_DIFF.subtract(1),1).map(lambda day: START.advance(day,'day'));

    #  /* Working resolution, minimum 30 meters, read
    #                          https://developers.google.com/earth-engine/guides/scale
    #                          to learn how scales/images pyramids work in GEE */

    eightCo = True;      # // eightConnected boolean for vectorization
    nCo = (eightCo+1)*4; #  // NConnected integer for file exports

    #/* int or float, multiplied by LANDSAT_SCALE, becomes the tolerance 
    #                                  of the simplification of the water polygons */

    # Visulization parameters :

    NDWI_MIN = -0.3;
    NDWI_MAX = 0.3;

    NDVI_MIN = -0.5;
    NDVI_MAX = 0.5;

    CLOUD_PAL = ['gray', 'white'];
    NDWI_PAL = ['black','white','blue'];
    NDVI_PAL = ['black','white','green'];

    LANDSAT_FULL = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL'])
        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL'])));
    LANDSAT_FULL = (LANDSAT_FULL
                        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                        .merge(ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))   
                    );

    # print("CLOUD_COVER FILTER: ", CLOUD_FILTER);
    LANDSAT = (LANDSAT_FULL
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER)) # COVERAGE CHECK (CC) AT THE IMAGE LEVEL!
        .map(maskL8sr)
        .filterBounds(AOI));

    # Nelly's argument --- remove first threshold and vary second.
    
    # GET COVER IS THE IMAGE COVER (How does getCover() work? -- )
    LANDSAT = LANDSAT.map(lambda image: getCover(image, AOI, LANDSAT_SCALE)); # CC AT THE DGO LEVEL!
    LANDSAT = LANDSAT.filterMetadata('percCover', 'greater_than', CLOUD_FILTER_2); # DEFAULT 98
    LANDSAT = LANDSAT.map(addMNDWI);

    LANDSAT = LANDSAT.map(addIS_WATER_MNDWI);

    IS_WATER_MNDWI = IS_WATER_INDIC('MNDWI', LANDSAT);
    return AOI, LANDSAT, IS_WATER_MNDWI;

MIN_PIX = 2; # // To filter small water polygons
CLOUD_FILTER = 80;
CLOUD_FILTER_2 = 25;
LANDSAT_SCALE = 30;
SIMPLIFY_TOLERANCE_N = 2;
AOI, LANDSAT, IS_WATER_MNDWI = process_polygon(shp=lhasa_shp, dgo=1, START_DATE='2020-01-01', END_DATE='2020-01-30');

Map.addLayer(LANDSAT.first(), {"bands": ['B3', 'B2', 'B1'], "min": 9600, "max": 16677}, 'RGB');
Map.addLayer(lhasa_shp, {}, 'Lhasa River SHP');
Map.addLayer(IS_WATER_MNDWI.first(), {'palette': 'Reds'}, 'water MNDWI', True);
Map.zoom = 11
Map.center_object(lhasa_shp);

In [7]:
shp_path = './Lhasa/Lhasa_RC_DGO2km_updated.shp';
lhasa_shp = geemap.shp_to_ee(shp_path);
Map.zoom = 15;
Map.center_object(lhasa_shp);
Map

Map(bottom=54523.0, center=[29.651569731000073, 91.3219064102816], controls=(ZoomControl(options=['position', …

In [ ]:
def VECTORIZE_FOR_AOI(AOI):
    def VECTORIZE(image):
        DAY_VECTORS = image.reduceToVectors(**{
            "geometry" : AOI,
            "scale" : LANDSAT_SCALE,
            "eightConnected" : True,
            "maxPixels" : 1e12,
            "geometryType" : 'polygon'
        });
        return DAY_VECTORS.set('DATE_ACQUIRED',image.get('DATE_ACQUIRED'));
    return VECTORIZE;

# Add area info to filter out isolated pixels
def ADDPROPS_FEAT(feature): 
    return feature.set({'AREA' : feature.area(LANDSAT_SCALE)});

def WRAP_COL_FOR_AOI(AOI):
    def WRAP_COL(collection):
        collection = (ee.FeatureCollection(collection).map(ADDPROPS_FEAT)
            .filterMetadata('AREA', 'greater_than', MIN_PIX * LANDSAT_SCALE * LANDSAT_SCALE));
        time_start = collection.get('DATE_ACQUIRED');
        collection.map(lambda feature: feature.set({'DATE_ACQUIRED' : time_start}));
        geom = collection.geometry();
        geom_smooth = collection.geometry().simplify(**{'maxError' : SIMPLIFY_TOLERANCE_N*LANDSAT_SCALE});
        polygon = AOI.geometry(); #this is the polygon of the AOI/DGO
        polygon_smooth = AOI.geometry().simplify(**{'maxError' : SIMPLIFY_TOLERANCE_N*LANDSAT_SCALE})

        return ee.Feature(geom, {
            'TOTAL_PERIM' : geom.perimeter(LANDSAT_SCALE),
            'AREA' : geom.area(LANDSAT_SCALE),
            'TOTAL_PERIM_SMOOTH' : geom_smooth.perimeter(LANDSAT_SCALE),
            'AREA_SMOOTH' : geom_smooth.area(LANDSAT_SCALE),
            'AREA_POLYGON' : polygon_smooth.area(LANDSAT_SCALE), # this is the polygon of the AOI/DGO
            'SENSING_TIME' : collection.get('DATE_ACQUIRED')
        });
    return WRAP_COL;

def WATER_VEC(AOI, ImColl):
    WVEC = ImColl.map(VECTORIZE_FOR_AOI(AOI));
    return ee.FeatureCollection(WVEC.map(WRAP_COL_FOR_AOI(AOI)));


import os
RIVER_PROPERTIES = [];
CLOUD_FILTER = 80;
CLOUD_FILTER_2 = 98; # Default 98
FILE_NAME = "Lhasa/Lhasa_Sentinel_Data-CC-80.csv";
# TODO: Read DGO Count from shp 
# Do 22 Dgos
for i in range(1, 77):
    # TODO: Print using ipython notify (progress bar...)    
    RIVER_PROPERTIES = [];
    print("DGO:", i, "/", 77);
    AOI, LANDSAT, IS_WATER_MNDWI = process_polygon(shp=lhasa_shp, dgo=i, START_DATE='1980-01-01', END_DATE='2100-01-01');
    ISWV_MNDWI = WATER_VEC(AOI, IS_WATER_MNDWI);
    info = ISWV_MNDWI.getInfo(); # PROCESS AND PULL RESULT
    property_list = map(lambda x: {
            "ID": i,
            "Tile_ID": x['id'],
            **x['properties']
        }, info['features']);
    # Map over properties (calculate Braiding Index e.t.c.)
    RIVER_PROPERTIES += list(property_list);
    if(os.path.exists(FILE_NAME)):
        df = pd.read_csv(FILE_NAME, index_col=0);
        df = pd.concat([df, pd.DataFrame(RIVER_PROPERTIES)]);
    else:
        df = pd.DataFrame(RIVER_PROPERTIES);
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')];
    df.to_csv(FILE_NAME);    
print("Done!");

DGO: 1 / 77
DGO: 2 / 77
DGO: 3 / 77
DGO: 4 / 77
DGO: 5 / 77
DGO: 6 / 77
DGO: 7 / 77
DGO: 8 / 77
DGO: 9 / 77
DGO: 10 / 77
DGO: 11 / 77
DGO: 12 / 77
DGO: 13 / 77
DGO: 14 / 77
DGO: 15 / 77
DGO: 16 / 77
DGO: 17 / 77
DGO: 18 / 77
DGO: 19 / 77
DGO: 20 / 77
DGO: 21 / 77
DGO: 22 / 77
DGO: 23 / 77
DGO: 24 / 77
DGO: 25 / 77
DGO: 26 / 77
DGO: 27 / 77
DGO: 28 / 77
DGO: 29 / 77
DGO: 30 / 77
DGO: 31 / 77
DGO: 32 / 77
DGO: 33 / 77
DGO: 34 / 77
DGO: 35 / 77
DGO: 36 / 77
DGO: 37 / 77
DGO: 38 / 77
DGO: 39 / 77
DGO: 40 / 77
DGO: 41 / 77
DGO: 42 / 77


In [9]:
LANDSAT_FULL = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL'])
        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL'])));
LANDSAT_FULL = (LANDSAT_FULL
                    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                    .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                    .merge(ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))   
                );
START_DATE='1980-01-01';
END_DATE='2022-06-02';
CLOUD_FILTER = 80;
BATCH_SIZE = 100;

LANDSAT = (LANDSAT_FULL
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER)) # COVERAGE CHECK (CC) AT THE IMAGE LEVEL!
    .map(maskL8sr)
    .filterBounds(lhasa_shp));

image_list = LANDSAT.toList(LANDSAT.size().getInfo())

for image in range(image_list.length().getInfo()):
    image = ee.Image(image)
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    filename = f"LANDSAT_{date}.tif"
    geemap.ee_export_image(image, filename=filename, scale=90, file_per_band=True, out_dir="./out_dir/")
    #geemap.ee_export_image_collection(LANDSAT, out_dir="./out_dir/")
# geemap.ee_export_image(image, filename='test.tif', scale=90, file_per_band=True)

EEException: Date: Parameter 'value' is required.

In [ ]:
LANDSAT_FULL = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL'])
        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL'])));
LANDSAT_FULL = (LANDSAT_FULL
                    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                    .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                    .merge(ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                    .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))   
                );
START_DATE='1980-01-01';
END_DATE='1989-01-01';
CLOUD_FILTER = 80;

LANDSAT = (LANDSAT_FULL
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER)) # COVERAGE CHECK (CC) AT THE IMAGE LEVEL!
    .map(maskL8sr)
    .filterBounds(lhasa_shp));

geemap.ee_export_image_collection(LANDSAT, out_dir="./out_dir/")
# geemap.ee_export_image(image, filename='test.tif', scale=90, file_per_band=True)

In [ ]:
df = pd.read_csv("test.csv")
df['BRAIDING_INDEX'] = (df['TOTAL_PERIM_SMOOTH']/2)/2000; # Calculate braiding index
df

In [ ]:
df_c = df.copy();

fig, axs = plt.subplots(9, 9, figsize=(24, 18), constrained_layout=True);
axs = axs.flatten();
for k, v in enumerate(range(1, 77)):
    df_d = df_c[df_c["ID"] == v];
    axs[k].set_title(v);
    axs[k].plot(df_d["SENSING_TIME"], df_d["BRAIDING_INDEX"], '+');

In [ ]:
# For call:
# Maybe try to get things working for Nelly...
# Python 3.10: https://github.com/google/earthengine-api/issues/181
# Python 3.9: Works
# Modules:
# numpy, pandas, matplotlib
# For Windows: pipwin gdal, fiona
# geemap, geopandas, ee
# TODO List:
# Fix laptop (DONE)
# Fix ownCloud (DONE)
# Email Eric about structure/limits of M1 Report (DONE) [reponse]


# Barbara:
# Look into making algorithm faster (doing all 78 in one pass?) [Currently it takes 1H to run...]
    # MAYBE I can run multiple .info() in seperate threads(??) (does google allow multiple requests like this?)
    # Cannot map over DGOs in a shape in a single command...
    # PERHAPS modify process_polygon (make it general) move it out of the loop!
    # Do process_polygon on ALL DGOs first - we can REUSE the same water map accordingly
    # Remove coordinates from object before sending it to clientside
# Try to make mosaic to avoid overlapping values. (Look at google earth api)
# https://developers.google.com/earth-engine/guides/ic_composite_mosaic

# Run again to extract SENTINEL data (from high-res sattelites)
  # REF: Barbara's email
# [Comparison between Sentinel & Landsat Data]
# Support for downloading tiles from meta information --> to use for GIS fieldwork

# Nelly/Patrice:
# Document installation procedure
# Reorganise repository 
# Write README.md

# For Monday:
# - extract the 4 channel images for the 200 timeslots and share them with me [finer scale analysis?]
# - extracted cvs. + processed data with l1 optimisation method for 3 differet level of cloud threshold
